In [27]:
from abc import ABC
from pathlib import Path
from timeit import repeat

from codetf.models import load_model_pipeline

In [28]:
root_dir = Path.cwd()

In [32]:
class AbstractModel(ABC):
    def predict(self) -> str:
        raise NotImplementedError()
    
    def model_name() -> str:
        raise NotImplementedError()
    
class CodeTFModel(AbstractModel):
    def __init__(self, model_name: str, model_type: str, task: str) -> None:
        super().__init__()

        self._model = load_model_pipeline(model_name=model_name, model_type=model_type, task=task)
        self._model_name = model_name
        self._model_type = model_type
        self._task = task

    def predict(self) -> str:
        return self._model.predict(['def hello(): print("Hello world")'])
    
    def model_name(self) -> str:
        return f"{self._model_name}-{self._model_type}-{self._task}"

In [55]:
def test_runtime(model: AbstractModel):   
    file_path = root_dir / "data" / "runtime" / f"{model.model_name()}.csv"

    if file_path.exists():
        return

    def _predict():
        model.predict()

    with file_path.open("w") as f:
        f.writelines(f"{n}\n" for n in repeat(_predict, number=1, repeat=100))

In [56]:
test_runtime(CodeTFModel(model_name="codet5", model_type="base-multi-sum", task="pretrained"))

In [ ]:
test_runtime(CodeTFModel(model_name="codet5", model_type="base", task="sum_python"))